In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import altair as alt
import ipywidgets as widgets
from IPython.display import display, clear_output
from ipywidgets import HBox, VBox, Layout
import warnings
warnings.filterwarnings("ignore")

# dati ISTAT
data = pd.read_csv("DCSP_COLTIVAZIONI_22032024191007716.csv")

# Rimozione spazi bianchi dopo le stringhe della colonna 'Tipo dato'
data['Tipo dato'] = data['Tipo dato'].str.rstrip()

# Filtro 'Tipo dato' di interesse
data_filtered = data[data["Tipo dato"].isin(['superficie totale - ettari', 'superficie totale - are', 
                                           'superficie in produzione - ettari', 'produzione totale - quintali', 
                                           'produzione totale - migliaia di quintali', 'produzione raccolta - quintali',
                                            'unità foraggere', 'superficie in produzione - ettari'])]

# Filtro 'Territorio' di interesse
data_filtered = data_filtered[data_filtered["Territorio"].isin(['Italia', 'Liguria', 'Lombardia',
                                             'Trentino Alto Adige / Südtirol', 'Veneto', 
                                             'Friuli-Venezia Giulia', 'Emilia-Romagna', 
                                             'Toscana', 'Umbria', 
                                             'Marche', 'Lazio', 
                                             'Campania', 'Puglia',  
                                             'Calabria', 'Sicilia',
                                             'Sardegna', 'Abruzzo', 
                                             'Basilicata', 'Molise', 
                                             "Valle d'Aosta / Vallée d'Aoste", 
                                             'Bolzano / Bozen', 'Piemonte',
                                             'Sud', 'Isole'])]
                                             
data_filtered2 = data_filtered
column_of_interest = ['Tipo dato', 'Tipo di coltivazione', 'TIME', 'Value', 'Territorio']
data_filtered3 = data_filtered2[column_of_interest]

def my_theme():
    return {
        "config": {
            'background': 'transparent',
            "title": {
                #"font": "Lato",  # Sostituisci con il nome del font desiderato
                "fontSize": 20,
                "anchor": "start",  # posizione del titolo
                "color": "black"
            },
            "axis": {
                #"labelFont": "Lato",  # Sostituisci con il nome del font desiderato
                #"titleFont": "Lato",  # Sostituisci con il nome del font desiderato
                'labelFontSize': 12,  # Font size for axis labels
                'titleFontSize': 14   # Font size for axis titles
            }
        }
    }

# Registra e imposta il tema
alt.themes.register('my_theme', my_theme)
alt.themes.enable('my_theme')

In [3]:
dark_theme_custom = {
    'config': {
        'background': '#121212',  # Colore di sfondo del grafico
        'title': {
            'color': '#E0E0E0',  # Colore del titolo
            'fontSize': 20
        },
        'axis': {
            'domainColor': '#B0B0B0',
            'gridColor': '#333333',
            'tickColor': '#B0B0B0',
            'titleColor': '#E0E0E0',
            'labelColor': '#E0E0E0',
            'labelFontSize': 12,  # Font size for axis labels
            'titleFontSize': 14   # Font size for axis titles
        },
        'legend': {
            'labelColor': '#E0E0E0',
            'titleColor': '#E0E0E0'
        },
        'mark': {
            'color': '#1f77b4'  # Colore del segno
        }
    }
}

# Registra e applica il tema personalizzato
alt.themes.register('dark_theme_custom', lambda: dark_theme_custom)
alt.themes.enable('dark_theme_custom')

ThemeRegistry.enable('dark_theme_custom')

In [4]:
rows_to_delete = ['grano invernale e farro', 'grano primaverile e farro', 'orzo invernale', 'orzo primaverile',
                    'uve per vini dop', 'uve per vini igp', 'uve per altri vini (escluso dop e igp)', 'olive da tavola', 
                    'altre olive', 'olive da olio', 'limoni e lime acidi', 'champignons', 
                    'altri funghi coltivati diversi dagli champignons', 'vino D.O.C. e D.O.C.G.', 'vino I.G.T.',
                    'vino', 'vino da tavola', 'vino bianco da tavola', 'vino rosso e rosato da tavola', 'vino D:O:P:', 'vino D:O:P: bianco',
                    'vino D:O:P: rosso e rosato', 'vino I.G.P.', 'vino I.G.P. bianco', 'vino I.G.P. rosso e rosato', 'olio di oliva']

In [5]:
data_filtered3 = data_filtered3[~data_filtered3["Tipo di coltivazione"].isin(rows_to_delete)]

In [6]:
# Lista per raccogliere i nuovi dati
new_data = []

# Raggruppa per Territorio e Time
for (territorio, time, colt), group in data_filtered3.groupby(['Territorio', 'TIME', 'Tipo di coltivazione']):
    new_data.append(group)
    
    # Controlla se esiste un record per 'unità foraggere' con value non vuoto
    foraggio_value = group.loc[group['Tipo dato'] == 'unità foraggere', 'Value']
    if not foraggio_value.empty and time <= 2019:
        prod_mille_value = group.loc[group['Tipo dato'] == 'produzione totale - migliaia di quintali', 'Value']
        if not prod_mille_value.empty:
            prod_value = prod_mille_value.iloc[0] * 1000
            
            # Se esiste già una riga con 'Tipo dato' = 'produzione totale - quintali', aggiorna il valore
            if 'produzione totale - quintali' in group['Tipo dato'].values:
                group.loc[group['Tipo dato'] == 'produzione totale - quintali', 'Value'] = prod_value
            else:
                # Se non esiste, crea una nuova riga
                new_row = {
                    'Territorio': territorio,
                    'Tipo dato': 'produzione totale - quintali',
                    'Tipo di coltivazione': colt,
                    'TIME': time,
                    'Value': prod_value
                }
                new_data.append(pd.DataFrame([new_row]))
            # Se esiste già una riga con 'Tipo dato' = 'produzione raccolta - quintali', aggiorna il valore
            if 'produzione raccolta - quintali' in group['Tipo dato'].values:
                group.loc[group['Tipo dato'] == 'produzione raccolta - quintali', 'Value'] = prod_value
            else:
                # Se non esiste, crea una nuova riga
                new_row = {
                    'Territorio': territorio,
                    'Tipo dato': 'produzione raccolta - quintali',
                    'Tipo di coltivazione': colt,
                    'TIME': time,
                    'Value': prod_value
                }
                new_data.append(pd.DataFrame([new_row]))
    if not foraggio_value.empty and time >= 2020:
        prod = group.loc[group['Tipo dato'] == 'produzione raccolta - quintali', 'Value']
        if not prod.empty:
            prod_value = prod.iloc[0]
            #prod_mille_value = prod.iloc[0] / 1000
        
            # Se esiste già una riga con 'Tipo dato' = 'produzione totale - quintali', aggiorna il valore
            if 'produzione totale - quintali' in group['Tipo dato'].values:
                group.loc[group['Tipo dato'] == 'produzione totale - quintali', 'Value'] = prod_value
            else:
                # Se non esiste, crea una nuova riga
                new_row = {
                    'Territorio': territorio,
                    'Tipo dato': 'produzione totale - quintali',
                    'Tipo di coltivazione': colt,
                    'TIME': time,
                    'Value': prod_value
                }
                new_data.append(pd.DataFrame([new_row]))

# Concatenazione dei dati raccolti
result = pd.concat(new_data, ignore_index=True)

In [6]:
mais = result[result['Territorio'] == 'Italia']
mais2 = mais[mais['Tipo di coltivazione'] == 'mais ceroso'] 
mais3 = mais2[mais2['TIME'] == 2019] 
mais3

,Tipo dato,Tipo di coltivazione,TIME,Value,Territorio
51164,unità foraggere,mais ceroso,2019,4623770.0,Italia
51165,superficie in produzione - ettari,mais ceroso,2019,367322.0,Italia
51166,superficie totale - ettari,mais ceroso,2019,367422.0,Italia
51167,produzione totale - migliaia di quintali,mais ceroso,2019,186914.0,Italia
51168,produzione totale - quintali,mais ceroso,2019,186914000.0,Italia
51169,produzione raccolta - quintali,mais ceroso,2019,186914000.0,Italia


In [16]:
data_filtered3 = result
#data_filtered3.to_excel('data_filtered.xlsx', index=False)

In [11]:
# Togli produzione totale - migliaia di quintali perchè arriva solo fino al 2019
data_filtered3 = data_filtered3[data_filtered3['Tipo dato'] != 'produzione totale - migliaia di quintali']

# Rinomina colonne di interesse
data_filtered3.rename(columns={'TIME': 'Anno', 'Value': 'Valore'}, inplace=True)

# Calcola valore totale per Tipo dato e TIME
summarized_df = data_filtered3.groupby(['Tipo dato', 'Anno', 'Territorio'])['Valore'].sum().reset_index()

# Rimuovi anno 2024 perchè mancano dei dati
summarized_df = summarized_df[summarized_df['Anno'] != 2024]

# Rimuovi unità foraggere
summarized_df = summarized_df[summarized_df['Tipo dato'] != 'unità foraggere']

# Seleziona solo Italia
summarized_df = summarized_df[summarized_df['Territorio'] == 'Italia']

# Crea una nuova colonna per concatenare il titolo
summarized_df['Title'] = summarized_df['Tipo dato'] + ' - ' + summarized_df['Territorio']

# Menù a tendina per "Tipo dato"
dropdown_options1 = sorted(summarized_df['Tipo dato'].unique().tolist())

# Selezione menù a tendina
dropdown1 = alt.binding_select(options=dropdown_options1, name = ' ')
selection1 = alt.selection_point(fields=['Tipo dato'], bind=dropdown1, name='SelectTipoDato', value = dropdown_options1[0])

#min_max_values = summarized_df.groupby(['Tipo dato', 'Territorio'])['Value'].agg(['min', 'max']).reset_index()

# Grafico a dispersione con cerchi
chart = alt.Chart(summarized_df).transform_filter(
    selection1 #& selection2
).mark_circle().encode(
    y='Valore:Q',
    x='Anno:N',
    size=alt.Size(field='Valore', type='quantitative', bin=alt.BinParams(maxbins=5)),
    color=alt.Color('Valore:Q', scale=alt.Scale(scheme='blues')),
    tooltip=['TIME', 'Valore']
).encode(
    x=alt.X('Anno:N', title='Anno', axis=alt.Axis(labelAngle=0, labelAlign='center', labelBaseline='top')),
    y=alt.Y('Valore:Q', title='Valore'),
    size=alt.Size('Valore:Q', scale=alt.Scale(range=[20, 300]), legend = None),
    color=alt.Color('Valore:Q', scale=alt.Scale(range=["#99f4be", "#18E169"])),
    tooltip=[alt.Tooltip('Anno:N'),
             alt.Tooltip('Valore:Q', format=',')]
    #tooltip=['Anno:N', 'Valore:Q']
).properties(
    width=600,
    #width = 'container',
    height=400
)

# Grafico a linee
line = alt.Chart(summarized_df).transform_filter(
    selection1
).mark_line(
    color="#188FE1",
    interpolate='monotone'
).encode(
    x=alt.X('Anno:N', title='Anno'),
    y=alt.Y('Valore:Q', title='Valore', scale=alt.Scale(zero=False)),
)

# Text chart per titolo dinamico
text = alt.Chart(summarized_df).transform_filter(
    selection1
).mark_text(
    align='center',
    dy=-15,
    fontSize=20
).encode(
    text='Title:N'
).properties(
    width=600,
    #width = 'container',
    height=30 
)

# Combina text chart and main chart
combined_chart = alt.vconcat(text, alt.layer(line, chart).add_selection(selection1))

# Salva il grafico in un file JSON
chart_json = combined_chart.to_json()
with open('chart_istat_smooth.json', 'w') as f:
    f.write(chart_json)

# Visualizza il grafico combinato
combined_chart

alt.VConcatChart(...)

In [9]:
# Salva il grafico in un file JSON
chart_json = chart.to_json()
with open(r'E:\Gianluca\Master Big Data Pisa\Progetto_Finale\Sito\g6-2024-website\assets\charts\chart_ric_dark_bg\chart_istat.json', 'w') as f:
    f.write(chart_json)

In [13]:
# Togli produzione totale - migliaia di quintali perchè arriva solo fino al 2019
data_filtered3 = data_filtered3[data_filtered3['Tipo dato'] != 'produzione totale - migliaia di quintali']

rows_to_delete = ['superficie totale - ettari', 'superficie totale - are', 'superficie in produzione - ettari']

data_filtered3 = data_filtered3[~data_filtered3["Tipo dato"].isin(rows_to_delete)]

# Rinomina colonne di interesse
data_filtered3.rename(columns={'TIME': 'Anno', 'Value': 'Valore'}, inplace=True)

# Calcola valore totale per Tipo dato e TIME
summarized_df = data_filtered3.groupby(['Tipo dato', 'Anno', 'Territorio'])['Valore'].sum().reset_index()

# Rimuovi anno 2024 perchè mancano dei dati
summarized_df = summarized_df[summarized_df['Anno'] != 2024]

# Rimuovi unità foraggere
summarized_df = summarized_df[summarized_df['Tipo dato'] != 'unità foraggere']

# Seleziona solo Italia
summarized_df = summarized_df[summarized_df['Territorio'] == 'Italia']

# Crea una nuova colonna per concatenare il titolo
summarized_df['Title'] = summarized_df['Tipo dato'] + ' - ' + summarized_df['Territorio']

# Menù a tendina per "Tipo dato"
dropdown_options1 = sorted(summarized_df['Tipo dato'].unique().tolist())

# Selezione menù a tendina
dropdown1 = alt.binding_select(options=dropdown_options1, name = ' ')
selection1 = alt.selection_point(fields=['Tipo dato'], bind=dropdown1, name='SelectTipoDato', value = dropdown_options1[0])

#min_max_values = summarized_df.groupby(['Tipo dato', 'Territorio'])['Value'].agg(['min', 'max']).reset_index()

# Grafico a dispersione con cerchi
chart = alt.Chart(summarized_df).transform_filter(
    selection1 #& selection2
).mark_circle().encode(
    y='Valore:Q',
    x='Anno:N',
    size=alt.Size(field='Valore', type='quantitative', bin=alt.BinParams(maxbins=5)),
    color=alt.Color('Valore:Q', scale=alt.Scale(scheme='blues')),
    tooltip=['TIME', 'Valore']
).encode(
    x=alt.X('Anno:N', title='Anno', axis=alt.Axis(labelAngle=0, labelAlign='center', labelBaseline='top')),
    y=alt.Y('Valore:Q', title='Valore'),
    size=alt.Size('Valore:Q', scale=alt.Scale(range=[20, 300]), legend = None),
    color=alt.Color('Valore:Q', scale=alt.Scale(range=["#99f4be", "#18E169"])),
    tooltip=[alt.Tooltip('Anno:N'),
             alt.Tooltip('Valore:Q', format=',')]
    #tooltip=['Anno:N', 'Valore:Q']
).properties(
    width=600,
    #width = 'container',
    height=400
)

# Grafico a linee
line = alt.Chart(summarized_df).transform_filter(
    selection1
).mark_line(
    color="#188FE1",
    interpolate='monotone'
).encode(
    x=alt.X('Anno:N', title='Anno'),
    y=alt.Y('Valore:Q', title='Valore', scale=alt.Scale(zero=False)),
)

# Text chart per titolo dinamico
text = alt.Chart(summarized_df).transform_filter(
    selection1
).mark_text(
    align='center',
    dy=-15,
    fontSize=20
).encode(
    text='Title:N'
).properties(
    width=600,
    #width = 'container',
    height=30 
)

# Combina text chart and main chart
combined_chart = alt.vconcat(text, alt.layer(line, chart).add_selection(selection1))

# Salva il grafico in un file JSON
chart_json = combined_chart.to_json()
with open('chart_istat_smooth.json', 'w') as f:
    f.write(chart_json)

# Visualizza il grafico combinato
combined_chart

alt.VConcatChart(...)

In [14]:
# Salva il grafico in un file JSON
chart_json = chart.to_json()
with open('chart_istat_prod.json', 'w') as f:
    f.write(chart_json)

In [17]:
# Togli produzione totale - migliaia di quintali perchè arriva solo fino al 2019
data_filtered3 = data_filtered3[data_filtered3['Tipo dato'] != 'produzione totale - migliaia di quintali']

rows_to_delete = ['produzione totale - quintali', 'produzione raccolta - quintali']

data_filtered3 = data_filtered3[~data_filtered3["Tipo dato"].isin(rows_to_delete)]

# Rinomina colonne di interesse
data_filtered3.rename(columns={'TIME': 'Anno', 'Value': 'Valore'}, inplace=True)

# Calcola valore totale per Tipo dato e TIME
summarized_df = data_filtered3.groupby(['Tipo dato', 'Anno', 'Territorio'])['Valore'].sum().reset_index()

# Rimuovi anno 2024 perchè mancano dei dati
summarized_df = summarized_df[summarized_df['Anno'] != 2024]

# Rimuovi unità foraggere
summarized_df = summarized_df[summarized_df['Tipo dato'] != 'unità foraggere']

# Seleziona solo Italia
summarized_df = summarized_df[summarized_df['Territorio'] == 'Italia']

# Crea una nuova colonna per concatenare il titolo
summarized_df['Title'] = summarized_df['Tipo dato'] + ' - ' + summarized_df['Territorio']

# Menù a tendina per "Tipo dato"
dropdown_options1 = sorted(summarized_df['Tipo dato'].unique().tolist())

# Selezione menù a tendina
dropdown1 = alt.binding_select(options=dropdown_options1, name = ' ')
selection1 = alt.selection_point(fields=['Tipo dato'], bind=dropdown1, name='SelectTipoDato', value = dropdown_options1[0])

#min_max_values = summarized_df.groupby(['Tipo dato', 'Territorio'])['Value'].agg(['min', 'max']).reset_index()

# Grafico a dispersione con cerchi
chart = alt.Chart(summarized_df).transform_filter(
    selection1 #& selection2
).mark_circle().encode(
    y='Valore:Q',
    x='Anno:N',
    size=alt.Size(field='Valore', type='quantitative', bin=alt.BinParams(maxbins=5)),
    color=alt.Color('Valore:Q', scale=alt.Scale(scheme='blues')),
    tooltip=['TIME', 'Valore']
).encode(
    x=alt.X('Anno:N', title='Anno', axis=alt.Axis(labelAngle=0, labelAlign='center', labelBaseline='top')),
    y=alt.Y('Valore:Q', title='Valore'),
    size=alt.Size('Valore:Q', scale=alt.Scale(range=[20, 300]), legend = None),
    color=alt.Color('Valore:Q', scale=alt.Scale(range=["#99f4be", "#18E169"])),
    tooltip=[alt.Tooltip('Anno:N'),
             alt.Tooltip('Valore:Q', format=',')]
    #tooltip=['Anno:N', 'Valore:Q']
).properties(
    width=600,
    #width = 'container',
    height=400
)

# Grafico a linee
line = alt.Chart(summarized_df).transform_filter(
    selection1
).mark_line(
    color="#188FE1",
    interpolate='monotone'
).encode(
    x=alt.X('Anno:N', title='Anno'),
    y=alt.Y('Valore:Q', title='Valore', scale=alt.Scale(zero=False)),
)

# Text chart per titolo dinamico
text = alt.Chart(summarized_df).transform_filter(
    selection1
).mark_text(
    align='center',
    dy=-15,
    fontSize=20
).encode(
    text='Title:N'
).properties(
    width=600,
    #width = 'container',
    height=30 
)

# Combina text chart and main chart
combined_chart = alt.vconcat(text, alt.layer(line, chart).add_selection(selection1))

# Salva il grafico in un file JSON
chart_json = combined_chart.to_json()
with open('chart_istat_smooth.json', 'w') as f:
    f.write(chart_json)

# Visualizza il grafico combinato
combined_chart

alt.VConcatChart(...)

In [ ]:
# Salva il grafico in un file JSON
chart_json = chart.to_json()
with open('chart_istat_sup.json', 'w') as f:
    f.write(chart_json)